In [ ]:
!pip install pycryptodome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.5 MB/s eta 0:00:00


In [ ]:
from Crypto.Cipher import AES
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
import os
import time

# **ENCRYPTION**

In [ ]:
# Generate an AES key and initialization vector (IV)
aes_key = os.urandom(32)
iv = os.urandom(16)

# Generate a new RSA key pair with a key size of 2048 bits
key = RSA.generate(2048)

# Save the private key to a file
with open("private_key.pem", "wb") as f:
    f.write(key.export_key())

# Save the public key to a file
with open("public_key.pem", "wb") as f:
    f.write(key.publickey().export_key())

In [ ]:
# Encrypt audio file with AES
t1=time.time()
with open("/content/file_example_WAV_10MG.wav", "rb") as f:
    audio_data = f.read()
    
cipher = AES.new(aes_key, AES.MODE_CFB, iv)
ciphertext = cipher.encrypt(audio_data)


# Encrypt the AES key and IV with RSA
public_key = RSA.import_key(open("public_key.pem").read())
cipher_rsa = PKCS1_OAEP.new(public_key)
encrypted_aes_key = cipher_rsa.encrypt(aes_key)
encrypted_iv = cipher_rsa.encrypt(iv)

# Save the encrypted audio file, encrypted AES key, and encrypted IV to disk
with open("encrypted_audio_file.wav", "wb") as f:
    f.write(ciphertext)

with open("encrypted_aes_key.bin", "wb") as f:
    f.write(encrypted_aes_key)

with open("encrypted_iv.bin", "wb") as f:
    f.write(encrypted_iv)

In [ ]:
#time taken

t2=time.time()
print('Time taken for encryption - ',(t2-t1)*1000,'ms')

Time taken for encryption -  531.6047668457031 ms


# **DECRYPTION**

In [ ]:
#Decryption

t3=time.time()
# Load the encrypted audio file, encrypted AES key, and encrypted IV from disk
with open("encrypted_audio_file.wav", "rb") as f:
    ciphertext = f.read()

with open("encrypted_aes_key.bin", "rb") as f:
    encrypted_aes_key = f.read()

with open("encrypted_iv.bin", "rb") as f:
    encrypted_iv = f.read()

# Decrypt the AES key and IV with RSA
private_key = RSA.import_key(open("private_key.pem").read())
cipher_rsa = PKCS1_OAEP.new(private_key)
aes_key = cipher_rsa.decrypt(encrypted_aes_key)
iv = cipher_rsa.decrypt(encrypted_iv)

# Decrypt the audio file with AES
cipher = AES.new(aes_key, AES.MODE_CFB, iv)
audio_data = cipher.decrypt(ciphertext)

# Save the decrypted audio file to disk
with open("decrypted_audio_file.wav", "wb") as f:
    f.write(audio_data)


In [ ]:
#time taken

t4=time.time()
print('Time taken for decryption - ',(t4-t3)*1000,'ms')

Time taken for decryption -  609.5767021179199 ms
